In [ ]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import keras
import KerasTools as KT
import ipywidgets
from io import BytesIO
import matplotlib.pyplot as plt
import skimage
import time

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
button_ly = ipywidgets.Layout(width='34px', height='30px', border='1px solid black')
bleft_st = ipywidgets.ButtonStyle(font_weight='bold', button_color='yellow')
bleft = ipywidgets.Button(layout=button_ly, style=bleft_st, icon='arrow-left')

bright_st = ipywidgets.ButtonStyle(font_weight='bold', button_color='yellow')
bright = ipywidgets.Button(layout=button_ly, style=bright_st, icon='arrow-right')

bstay_st = ipywidgets.ButtonStyle(font_weight='bold', button_color='yellow')
bstay = ipywidgets.Button(layout=button_ly, style=bstay_st, icon='dot-circle-o')

lt = ipywidgets.Layout(width='68px', height='30px', border='1px solid black')
stt = ipywidgets.ButtonStyle(font_weight='bold', button_color='green')
bnew = ipywidgets.Button(layout=lt, style=stt, description = 'New')
stat = ipywidgets.HTML(layout=lt)

bx = ipywidgets.HBox([bnew, bleft, bstay, bright, stat])

img = ipywidgets.Image()
game = KT.qlearn.catch.Catch()

def plot_game(frame):
    imbuf = BytesIO()
    f = skimage.transform.resize(frame,(256,256), order=0, mode='constant', anti_aliasing=False)
    plt.imsave(imbuf, f, vmin=0.0, vmax=1.0, cmap='nipy_spectral')
    imbuf.seek(0)           
    img.value = imbuf.getvalue()

def onButtonClicked(args):
    if args.description == 'New':
        game.reset()
        stat.value = ""
    elif not game.is_over():
        args.style.button_color = 'red'
        time.sleep(0.1)
        args.style.button_color = 'yellow'
        action = {'arrow-left': 0, 'dot-circle-o': 1, 'arrow-right': 2}
        game.play(action[args.icon])
    stat.value = "<center><big><b>R: {}".format(str(game.get_score()))    
    plot_game(game.get_frame())

bleft.on_click(onButtonClicked)
bstay.on_click(onButtonClicked)
bright.on_click(onButtonClicked)
bnew.on_click(onButtonClicked)

display(img)
display(bx)
plot_game(game.get_frame())

In [ ]:
model = keras.models.load_model('catch_gru.h5')
model.summary()
game = KT.qlearn.catch.Catch()
agent = KT.qlearn.agent.Agent(model=model, nb_frames = 10)

In [ ]:
button_map = {0: bleft, 1: bstay, 2: bright}
num_games = 10
won = 0
for i in range(num_games):
    while not game.is_over():
        time.sleep(0.1)
        action = agent.get_action(game)
        button_map[action].click()
    time.sleep(1)
    if game.is_won(): won += 1
    bnew.click()
print("Won {:2.0%} of games".format(float(won)/num_games))

In [ ]:
model = keras.models.load_model('tromis_7x12_130.h5')
model.summary()
game = KT.qlearn.tromis.Tromis(width=7,height=12,max_turn=None)
agent = KT.qlearn.agent.Agent(model=model, nb_frames = 12)

In [ ]:
button_map = {0: bleft, 1: bright, 2: bstay, 3:brotleft, 4:brotright}
num_games = 10
won = 0
for i in range(num_games):
    while not game.is_over():
        time.sleep(0.1)
        action = agent.get_action(game)
        button_map[action].click()
    time.sleep(1)
    if game.is_won(): won += 1
    bnew.click()
print("Won {:2.0%} of games".format(float(won)/num_games))